In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Setting the current working directory
import os; os.chdir('/content/drive/MyDrive/AI_ML/Projects/Capstone-NLP')

### Importing the Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
%tensorflow_version 1.x

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import keras.layers as layers
from keras.models import Model
from keras import backend as K
np.random.seed(10)

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
#print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  1.15.2
Eager mode:  False
Hub version:  0.11.0


In [ ]:
#Function 1.1 - Load the dataset

def load_dataset(filename):
  data = pd.read_csv("IHMStefanini_industrial_safety_and_health_database_with_accidents_description.csv")
  return data

#Function 1.2 - Dataset Cleansing

def data_cleansing_info(data):
  data.drop("Unnamed: 0", axis=1, inplace=True)
  data.rename(columns={'Data':'Date', 'Countries':'Country', 'Accident Level' : 'AccLevel' ,  'Genre':'Gender', 'Employee or Third Party':'Employee type'}, inplace=True)
  return data.head(5), data.isnull().sum()

In [ ]:
data = load_dataset("IHMStefanini_industrial_safety_and_health_database_with_accidents_description.csv")
data.head()

,Unnamed: 0,Data,Countries,Local,Industry Sector,Accident Level,Potential Accident Level,Genre,Employee or Third Party,Critical Risk,Description
0,0,1/1/2016 0:00,Country_01,Local_01,Mining,I,IV,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...
1,1,1/2/2016 0:00,Country_02,Local_02,Mining,I,IV,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...
2,2,1/6/2016 0:00,Country_01,Local_03,Mining,I,III,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...
3,3,1/8/2016 0:00,Country_01,Local_04,Mining,I,I,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...
4,4,1/10/2016 0:00,Country_01,Local_04,Mining,IV,IV,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...


In [ ]:
data_cleansing_info(data)

(             Date  ...                                        Description
 0   1/1/2016 0:00  ...  While removing the drill rod of the Jumbo 08 f...
 1   1/2/2016 0:00  ...  During the activation of a sodium sulphide pum...
 2   1/6/2016 0:00  ...  In the sub-station MILPO located at level +170...
 3   1/8/2016 0:00  ...  Being 9:45 am. approximately in the Nv. 1880 C...
 4  1/10/2016 0:00  ...  Approximately at 11:45 a.m. in circumstances t...
 
 [5 rows x 10 columns], Date                        0
 Country                     0
 Local                       0
 Industry Sector             0
 AccLevel                    0
 Potential Accident Level    0
 Gender                      0
 Employee type               0
 Critical Risk               0
 Description                 0
 dtype: int64)

In [ ]:
data.head() #Column names corrected

,Date,Country,Local,Industry Sector,AccLevel,Potential Accident Level,Gender,Employee type,Critical Risk,Description
0,1/1/2016 0:00,Country_01,Local_01,Mining,I,IV,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...
1,1/2/2016 0:00,Country_02,Local_02,Mining,I,IV,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...
2,1/6/2016 0:00,Country_01,Local_03,Mining,I,III,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...
3,1/8/2016 0:00,Country_01,Local_04,Mining,I,I,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...
4,1/10/2016 0:00,Country_01,Local_04,Mining,IV,IV,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...


In [ ]:
replace_values = {'Local_01': 1, 'Local_02': 2, 'Local_03': 3, 'Local_04': 4, 'Local_05': 5, 'Local_06': 6, 'Local_07': 7, 'Local_08': 8, 'Local_09': 9, 'Local_10': 10, 'Local_11': 11, 'Local_12': 12}
data['Local'] = data['Local'].map(replace_values)
replace_values = {'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5}
#data['Accident Level'] = data['Accident Level'].map(replace_values)
replace_values = {'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5, 'VI': 6}
data['Potential Accident Level'] = data['Potential Accident Level'].map(replace_values)
del replace_values

In [ ]:
print(f'Dataset after adjusting categorical values\n')
data.head()

Dataset after adjusting categorical values



,Date,Country,Local,Industry Sector,AccLevel,Potential Accident Level,Gender,Employee type,Critical Risk,Description
0,1/1/2016 0:00,Country_01,1,Mining,I,4,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...
1,1/2/2016 0:00,Country_02,2,Mining,I,4,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...
2,1/6/2016 0:00,Country_01,3,Mining,I,3,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...
3,1/8/2016 0:00,Country_01,4,Mining,I,1,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...
4,1/10/2016 0:00,Country_01,4,Mining,IV,4,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...


In [ ]:
data.shape

(425, 10)

In [ ]:
df1 = data[['AccLevel', 'Description']]
df1.head()

,AccLevel,Description
0,I,While removing the drill rod of the Jumbo 08 f...
1,I,During the activation of a sodium sulphide pum...
2,I,In the sub-station MILPO located at level +170...
3,I,Being 9:45 am. approximately in the Nv. 1880 C...
4,IV,Approximately at 11:45 a.m. in circumstances t...


In [ ]:
import pickle

with open('df1.pickle', 'wb') as to_write:
  pickle.dump(df1, to_write)

In [ ]:
# Exploring the number of mutli-class categories we have
print('--------------')
print('Categories in Accident Level column:')
print('--------------\n')
print(df1['AccLevel'].unique(), '\n')
print('--------------')
print('# of unique categories: ', df1['AccLevel'].nunique())
print('--------------')

--------------
Categories in Accident Level column:
--------------

['I' 'IV' 'III' 'II' 'V'] 

--------------
# of unique categories:  5
--------------


In [ ]:
 df1.head()

,AccLevel,Description
0,I,While removing the drill rod of the Jumbo 08 f...
1,I,During the activation of a sodium sulphide pum...
2,I,In the sub-station MILPO located at level +170...
3,I,Being 9:45 am. approximately in the Nv. 1880 C...
4,IV,Approximately at 11:45 a.m. in circumstances t...


In [ ]:
df2 = df1.copy()

###First 5 Rows look like this.

In [ ]:
df1.head()

,AccLevel,Description
0,I,While removing the drill rod of the Jumbo 08 f...
1,I,During the activation of a sodium sulphide pum...
2,I,In the sub-station MILPO located at level +170...
3,I,Being 9:45 am. approximately in the Nv. 1880 C...
4,IV,Approximately at 11:45 a.m. in circumstances t...


In [ ]:
# NLP Import packages
import nltk; nltk.download('wordnet'); nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import string

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import re
from string import punctuation
from wordcloud import STOPWORDS

stop_words =  stopwords.words('english')

def text_cleaning(text, remove_stop_words=True, lemmatize_words=True):
    # Clean the text, with the option to remove stop_words and to lemmatize word

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r"ur", " your ", text)
    text = re.sub(r" nd "," and ",text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r" tkts "," tickets ",text)
    text = re.sub(r" c "," can ",text)
    text = re.sub(r" e g ", " eg ", text)
    text =  re.sub(r'http\S+',' link ', text)
    text = re.sub(r'\b\d+(?:\.\d+)?\s+', '', text) # remove numbers
    text = re.sub(r" u "," you ",text)
    text = text.lower()  # set in lowercase 
        
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if lemmatize_words:
        text = text.split()
        lemmatizer = WordNetLemmatizer() 
        lemmatized_words = [lemmatizer.lemmatize(word) for word in text]
        text = " ".join(lemmatized_words)
    
    # Return a list of words
    return(text)

##Cleaning the col "Description" inplace

In [ ]:
#clean the dataset 
#new_data["clean_message"] = new_data["message"].apply(text_cleaning)
df1["clean_Description"] = df1["Description"].apply(text_cleaning)
df1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,AccLevel,Description,clean_Description
0,I,While removing the drill rod of the Jumbo 08 f...,removing drill rod jumbo maintenance superviso...
1,I,During the activation of a sodium sulphide pum...,ing activation sodium sulphide pump piping unc...
2,I,In the sub-station MILPO located at level +170...,sub station milpo located level collaborator e...
3,I,Being 9:45 am. approximately in the Nv. 1880 C...,approximately nv cx ob7 personnel begin task u...
4,IV,Approximately at 11:45 a.m. in circumstances t...,approximately circumstance mechanic anthony gr...
...,...,...,...
420,I,"Being approximately 5:00 a.m. approximately, w...",approximately approximately lifting kelly hq t...
421,I,The collaborator moved from the infrastructure...,collaborator moved infrastruct e office julio ...
422,I,During the environmental monitoring activity i...,ing environmental monitoring activity area emp...
423,I,The Employee performed the activity of strippi...,employee performed activity stripping cathode ...


####Drop col "Description" & Rename clean_Description to Description

In [ ]:
df1.drop('Description', axis=1, inplace=True)
df1.rename(columns={'clean_Description':'Description'}, inplace=True)
df1.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,AccLevel,Description
0,I,removing drill rod jumbo maintenance superviso...
1,I,ing activation sodium sulphide pump piping unc...
2,I,sub station milpo located level collaborator e...
3,I,approximately nv cx ob7 personnel begin task u...
4,IV,approximately circumstance mechanic anthony gr...


In [ ]:
df1['AccLevel'] = df1['AccLevel'].astype('category')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
cat_counts = len(df1.AccLevel.cat.categories)
cat_counts

5

###**We want our model to be a multiclass classification model that takes strings as input and output probability for each of the 5 class labels.**

In [ ]:
train.shape, test.shape

((340, 2), (85, 2))

In [ ]:
train_Description = train['Description'].tolist()
train_Description = np.array(train_Description, dtype=object)[:, np.newaxis]
train_Description

array([['moving roll electro welded mesh kg weight place hopper truck two operator pull roll mesh bring hopper truck time imprisonment left hand one operator mesh body hopper generating described inj time accident operator wearing pvc type safety glove'],
       ['raise chamber operator assistant removed drilling bar 71r drilling machine weight approx kg holding top tm stroke assisted jib move horizontally operator jib bottom slide moving top drill bar causing assistant bar enforce assistant hand bar piston equipment'],
       ['ing die cutting feeder pead geomembrane blanket weld seam store extruder stylet blade came direction left forearm resulting blunt short inj'],
       ['performing sleeve removal maneuver hole meter deep general da silva pressed one side locking nut rod together jack hold entire weight rod maneuver locking proced e effective weight rod sec ed steel wire rope probe winch moment driller pedro released brake winch inefficacy locking done one side chestnut without a

##Perform Train/Test Split on Dataset

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df1, test_size = 0.2, random_state = 123, shuffle = True)

In [ ]:
train_AccLevel = np.asarray(pd.get_dummies(train.AccLevel), dtype = np.int8)
train_AccLevel.shape

(340, 5)

In [ ]:
test_Decription = test['Description'].tolist()
test_Decription = np.array(test_Decription, dtype=object)[:, np.newaxis]
test_Decription.shape

(85, 1)

In [ ]:
test_AccLevel = np.asarray(pd.get_dummies(test.AccLevel), dtype = np.int8)
test_AccLevel.shape

(85, 5)

##Load the Universal Sentence Encoder as variable "embed"

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [ ]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

In [ ]:
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value
embed_size

512

##Build the Keras model

###Wrap embed module in a Lambda layer
Explicitly cast the input as a string

In [ ]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [ ]:
#input_text = layers.Input(shape=(1,), dtype=tf.string)
#embedding = layers.Lambda(UniversalEmbedding, output_shape=(embed_size,))(input_text)
#dense = layers.Dense(256, activation='relu')(embedding)
#pred = layers.Dense(cat_counts, activation='softmax')(dense)
#model = Model(inputs=[input_text], outputs=pred, trainable = False)

In [ ]:
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Flatten, Bidirectional, GlobalMaxPool1D,GlobalAveragePooling1D

input_text = layers.Input(shape=(512,), dtype=tf.string)
embedding = layers.Lambda(UniversalEmbedding, output_shape=(embed_size,))(input_text)
#lst = layers.LSTM(units=128 , return_sequences = True, recurrent_dropout = 0.5 , dropout = 0.5)(embedding)
dense = layers.Dense(256, activation='relu')(embedding)
pred = layers.Dense(cat_counts, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred, trainable = True)

##Compile the Model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

##View the model summary

In [ ]:
model.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 512)               0         
_________________________________________________________________
lambda_16 (Lambda)           (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_24 (Dense)             (None, 5)                 1285      
Total params: 132,613
Trainable params: 132,613
Non-trainable params: 0
_________________________________________________________________


###Train Keras model and save weights
- Train the model with the training datasets
- Validate its performance at the end of each training epoch with test datasets.
- This only train and save our Keras layers not the embed module' weights.

In [ ]:
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(train_Description, 
            train_AccLevel,
            validation_data=(test_Decription, test_AccLevel),
            epochs=10,
            batch_size=32)
  model.save_weights('./model.h5')

ValueError: ignored

In [ ]:
!ls -alh | grep model.h5

##Make predictions
We have the model trained and its weights saved to a file

In [ ]:
#new_text = ["In what year did the titanic sink ?", "What is the highest peak in California ?", "Who invented the light bulb ?"]
new_text = ["The collaborator reports that he was on street 09 holding in his left hand the volumetric balloon ?"]
new_text = np.array(new_text, dtype=object)[:, np.newaxis]
print(new_text)

#embed5 = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
embeddings = embed(new_text)
print(embeddings)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

In [ ]:
cat_counts = train.AccLevel.cat.categories.tolist()
print(cat_counts)

with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(new_text, batch_size=32)

print(predicts)

predict_logits = predicts.argmax(axis=1)
predict_labels = [cat_counts[logit] for logit in predict_logits]
predict_labels

In [ ]:
message_embeddings.info

In [ ]:
from keras import backend as K

# Compute a representation for each message, showing various lengths supported.
#word = "Elephant"
#sentence = "I am a sentence for which I would like to get its embedding."
sentence = ["In the sub-station MILPO located at level +170 when the collaborator was doing the excavation work with a pick (hand tool), hitting a rock with the flat part of the beak, it bounces off hitting the steel tip of the safety shoe and then the metatarsal area of â€‹â€‹the left foot of the collaborator causing the injury."]
sentence = np.array(sentence, dtype=object)[:, np.newaxis]
#paragraph = (
#    "Universal Sentence Encoder embeddings also support short paragraphs. "
#    "There is no hard limit on how long the paragraph is. Roughly, the longer "
#    "the more 'diluted' the embedding will be.")
#messages = [word, sentence, paragraph]
messages = [sentence]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

#input_text = layers.Input(shape=(1,), dtype=tf.string)

with tf.Session() as session:
  K.set_session(session)
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(sentence))
  # print(message_embeddings.shape)
  # print(message_embedding)
  # #message_embeddings = layers.Input(shape=(1,), dtype=tf.string)
  # #print(message_embedding)
  # #shape = K.shape(message_embeddings) # tensor shape
  # #print(shape)
  # for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
  #   print("Message: {}".format(messages[i]))
  #   print("Embedding size: {}".format(len(message_embedding)))
  #   message_embedding_snippet = ", ".join(
  #       (str(x) for x in message_embedding[:3]))
  #   print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

  # #message_embeddings = layers.Input(shape=(1,), dtype=tf.string)
  model.load_weights('./model.h5')
  predicts = model.predict(message_embeddings, batch_size=32)

#with tf.Session() as session:
#  K.set_session(session)
#  session.run(tf.global_variables_initializer())
#  session.run(tf.tables_initializer())
#  model.load_weights('./model.h5')  
#  predicts = model.predict(new_text, batch_size=32)

print(predicts)



TypeError: ignored